# Cálculo de superficie municipal y densidad de explotaciones agrarias y ganaderas (España, 2020)


## 1. Importar librerías necesarias


In [ ]:
# Librerías básicas de QGIS
from qgis.core import *
from PyQt5.QtCore import QVariant
import processing

In [ ]:
# Comprobación
# Listar TODOS los nombres de capas cargadas en el proyecto
for lyr in QgsProject.instance().mapLayers().values():
    print(lyr.name())

## 2. Cargar las capas originales del proyecto

In [ ]:
# Capa de explotaciones agrarias
agr_layer = QgsProject.instance().mapLayersByName("NumExpAgr_2020")[0]

# Capa de explotaciones ganaderas
gan_layer = QgsProject.instance().mapLayersByName("NumExpGan_2020")[0]


## 3. Reproyectar capas a un SRC en metros

In [ ]:
# Reproyectar la capa de explotaciones AGRARIAS a ETRS89 / UTM 30N
agr_utm = processing.run(
    "native:reprojectlayer",
    {
        'INPUT': agr_layer,
        'TARGET_CRS': QgsCoordinateReferenceSystem('EPSG:25830'),
        'OUTPUT': 'memory:agr_utm'
    }
)['OUTPUT']

# Reproyectar la capa de explotaciones GANADERAS a ETRS89 / UTM 30N
gan_utm = processing.run(
    "native:reprojectlayer",
    {
        'INPUT': gan_layer,
        'TARGET_CRS': QgsCoordinateReferenceSystem('EPSG:25830'),
        'OUTPUT': 'memory:gan_utm'
    }
)['OUTPUT']


## 4. Funciónn para calcular superficie municipal en hectáreas

In [ ]:
def calcular_area_ha(layer):
    """
    Calcula el área de cada municipio en hectáreas (ha)
    y la guarda en un campo llamado 'ha'
    """
    with edit(layer):

        # Crear el campo 'ha' si no existe
        if layer.fields().indexFromName('ha') == -1:
            layer.addAttribute(QgsField('ha', QVariant.Double))

        idx_ha = layer.fields().indexFromName('ha')

        # Calcular área para cada entidad
        for f in layer.getFeatures():
            area_ha = f.geometry().area() / 10000  # m² → hectáreas
            layer.changeAttributeValue(f.id(), idx_ha, area_ha)


In [ ]:
# Calcular superficie municipal en ambas capas reproyectadas
calcular_area_ha(agr_utm)
calcular_area_ha(gan_utm)


## 5. Función para calcular densidad de explotaciones

In [ ]:
def calcular_densidad(layer, campo_explotaciones, campo_densidad):
    """
    Calcula la densidad de explotaciones:
    densidad = nº de explotaciones / hectáreas
    """
    with edit(layer):

        # Crear el campo de densidad si no existe
        if layer.fields().indexFromName(campo_densidad) == -1:
            layer.addAttribute(QgsField(campo_densidad, QVariant.Double))

        idx_ha = layer.fields().indexFromName('ha')
        idx_exp = layer.fields().indexFromName(campo_explotaciones)
        idx_dens = layer.fields().indexFromName(campo_densidad)

        for f in layer.getFeatures():
            ha = f[idx_ha]
            exp = f[idx_exp]

            # Evitar divisiones por cero o valores nulos
            dens = exp / ha if ha and ha > 0 else None
            layer.changeAttributeValue(f.id(), idx_dens, dens)


In [ ]:
# Densidad de explotaciones agrarias
calcular_densidad(
    agr_utm,
    campo_explotaciones='n_exp_agr',
    campo_densidad='dens_agr'
)

# Densidad de explotaciones ganaderas
calcular_densidad(
    gan_utm,
    campo_explotaciones='n_exp_gan',
    campo_densidad='dens_gan'
)


## 6. Copiar resultados a las capas originales (EPSG:4258)

In [ ]:
def copiar_campos(origen, destino, campos):
    """
    Copia valores de una capa temporal reproyectada
    a la capa original, campo a campo
    """
    with edit(destino):

        # Crear los campos en la capa original si no existen
        for campo in campos:
            if destino.fields().indexFromName(campo) == -1:
                destino.addAttribute(QgsField(campo, QVariant.Double))

        # Copiar valores entidad a entidad
        for f_origen, f_destino in zip(origen.getFeatures(), destino.getFeatures()):
            for campo in campos:
                idx = destino.fields().indexFromName(campo)
                destino.changeAttributeValue(
                    f_destino.id(),
                    idx,
                    f_origen[campo]
                )


In [ ]:
# Copiar superficie y densidad a las capas originales
copiar_campos(agr_utm, agr_layer, ['ha', 'dens_agr'])
copiar_campos(gan_utm, gan_layer, ['ha', 'dens_gan'])
